# 자동차 환경 빅데이터 3차
- 과제2 : 건설기계 배출가스 관리 체게 구축
- 과제4 : 배출가스 전문정비제도 관리체계 구축
- 과제5 : 자동차 환경 빅데이터 제작사 공동 활용체게 구축

# Library

In [39]:
import pandas as pd
import os
import numpy as np
from datetime import datetime

In [6]:
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# Fold

In [34]:
raw_fold = 'D:/data/processing/big3/raw'
raw2_fold = 'D:/data/processing/big3/raw/BD2'
raw4_fold = 'D:/data/processing/big3/raw/BD4'
raw5_fold = 'D:/data/processing/big3/raw/BD5'
an_fold = 'D:/data/processing/big3/analysis'
an2_fold = 'D:/data/processing/big3/analysis/BD2'
df_fold = 'D:/data/processing/big3/df'
df2_fold = 'D:/data/processing/big3/df/BD2'
df4_fold = 'D:/data/processing/big3/df/BD4'
df5_fold = 'D:/data/processing/big3/df/BD5'
folds = {'raw_fold':raw_fold, 'an_fold':an_fold, 'df_fold':df_fold, 
         'raw2_fold':raw2_fold, 'raw4_fold':raw4_fold, 'raw5_fold':raw5_fold, 
         'an2_fold:':an2_fold,
         'df2_fold':df2_fold, 'df4_fold':df4_fold, 'df5_fold':df5_fold}
folds

{'raw_fold': 'D:/data/processing/big3/raw',
 'an_fold': 'D:/data/processing/big3/analysis',
 'df_fold': 'D:/data/processing/big3/df',
 'raw2_fold': 'D:/data/processing/big3/raw/BD2',
 'raw4_fold': 'D:/data/processing/big3/raw/BD4',
 'raw5_fold': 'D:/data/processing/big3/raw/BD5',
 'an2_fold:': 'D:/data/processing/big3/analysis/BD2',
 'df2_fold': 'D:/data/processing/big3/df/BD2',
 'df4_fold': 'D:/data/processing/big3/df/BD4',
 'df5_fold': 'D:/data/processing/big3/df/BD5'}

# Load
+ 과제2
    - 건설기계 인증자료
    - 건설기계 인증대장
+ 과제4
    - 자동차전문정비 : STD_TB_ERR_RES
    - 자동차전문정비상세 : STD_TB_ERR_RES_DET
+ 과제5
    - 구조변경검사 : STD_DLM_TB_ERP_GUBUN_BAECHUL

## 과제2 > 건설기계 인증자료

In [81]:
def construct_machine_certification_data_load(name, sheet_name=0):
    try:
        file_name = f'{name}.xlsx'
        cert_dt = pd.read_excel(os.path.join(raw2_fold + '/KENCIS 건설기계 인증자료_231018', file_name), sheet_name=sheet_name)
        return cert_dt
    except:
        file_name = f'{name}.xlsx'
        cert_dt = pd.read_excel(os.path.join(raw2_fold + '/KENCIS 건설기계 인증자료_231018', file_name))
        return cert_dt

In [93]:
cols = construct_machine_certification_data_load('KENCIS_건설기계인증정보_테이블명세서', '테이블정의서')
cols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   테이블명    12 non-null     object
 1   테이블설명   12 non-null     object
 2   컬럼명     649 non-null    object
 3   컬럼설명    644 non-null    object
 4   데이터형식   649 non-null    object
 5   길이      499 non-null    object
 6   Null    649 non-null    object
dtypes: object(7)
memory usage: 35.6+ KB


In [94]:
cols[['테이블명', '테이블설명']] = cols[['테이블명', '테이블설명']].ffill()

In [35]:
name_list = [
    'UNI_FC_ENDUENG_EMI', 
    'UNI_FC_ENDUENGTEST', 
    'UNI_FC_ENDUENGTEST_RES', 
    'UNI_FC_ENG_LIST', 
    'UNI_FC_ENG_TEFSPEC', 
    'UNI_FC_KIENGTEST', 
    'UNI_FC_KIENGTEST_RES', 
    'UNI_FC_TCERTIMODI', 
    'UNI_FC_TCMASTER', 
    'UNI_FC_TEEMISSN', 
    'UNI_FC_TEEMISSN_ENG_RES', 
    'UNI_FC_TESTINFO'
    ]
len(name_list)

12

In [110]:
cols_dict = {}
for t in name_list:
    cols_dict[t] = {a:b for a, b in cols.loc[cols['테이블명'] == t, ['컬럼명', '컬럼설명']].values}

In [111]:
cols_dict

{'UNI_FC_ENDUENG_EMI': {'ADM_NO': '관리번호',
  'CERTI_MODI_SNO': '변경인증일련번호',
  'CO_GRAD': 'CO기울기',
  'CO_YITCT': 'CO Y절편',
  'MOD_DATETIME': '수정일시',
  'MOD_ID': '수정아이디',
  'NMHC_GRAD': 'NMHC기울기',
  'NMHC_YITCT': 'NMHC Y절편',
  'NOX_GRAD': 'NOX기울기',
  'NOX_YITCT': 'NOX Y절편',
  'PM_GRAD': 'PM기울기',
  'PM_YITCT': 'PM Y절편',
  'REG_DATETIME': '등록일시',
  'REG_ID': '등록아이디',
  'TEST_METHOD_CHK': '시험방법체크값',
  'TEST_MOD': '배출가스시험모드',
  'TEST_RST_SNO': '시험결과일련번호',
  'PN_GRAD': 'PN 기울기',
  'PN_YITCT': 'PN Y절편'},
 'UNI_FC_ENDUENGTEST': {'ADM_NO': '관리번호',
  'CERTI_MODI_SNO': '변경인증일련번호',
  'ENG_ID_NO': '대표원동기명(원동기코드)',
  'ENG_NO': '원동기 번호',
  'ENG_TYPE': '엔진형식',
  'MOD_DATETIME': '수정일시',
  'MOD_ID': '수정아이디',
  'REG_DATETIME': '등록일시',
  'REG_ID': '등록아이디',
  'TEST_DATE': '시험일',
  'TEST_ENG_SNO': '시험원동기일련번호',
  'TEST_NO': '시험번호'},
 'UNI_FC_ENDUENGTEST_RES': {'ADM_NO': '관리번호',
  'CERTI_MODI_SNO': '인증 일련번호',
  'P_ENG_YN': '내구대표차원동기여부',
  'SEL_ADM_NO': '내구 동일원동기  관리번호',
  'TEST_METHOD_CHK': '시험방법체크값',
  'TEST_MO

In [112]:
name = 'UNI_FC_ENDUENG_EMI'
ct01 = construct_machine_certification_data_load(name)
ct01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ADM_NO           411 non-null    int64  
 1   CERTI_MODI_SNO   411 non-null    int64  
 2   TEST_METHOD_CHK  411 non-null    object 
 3   TEST_MOD         411 non-null    int64  
 4   TEST_RST_SNO     411 non-null    int64  
 5   NOX_GRAD         167 non-null    float64
 6   NMHC_GRAD        153 non-null    float64
 7   CO_GRAD          163 non-null    float64
 8   PM_GRAD          166 non-null    float64
 9   NOX_YITCT        166 non-null    float64
 10  NMHC_YITCT       153 non-null    float64
 11  CO_YITCT         165 non-null    float64
 12  PM_YITCT         165 non-null    float64
 13  REG_ID           411 non-null    object 
 14  REG_DATETIME     375 non-null    object 
 15  MOD_ID           57 non-null     object 
 16  MOD_DATETIME     21 non-null     object 
 17  PN_GRAD         

c:\Users\kbjung\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [113]:
cols_dict[name]

{'ADM_NO': '관리번호',
 'CERTI_MODI_SNO': '변경인증일련번호',
 'CO_GRAD': 'CO기울기',
 'CO_YITCT': 'CO Y절편',
 'MOD_DATETIME': '수정일시',
 'MOD_ID': '수정아이디',
 'NMHC_GRAD': 'NMHC기울기',
 'NMHC_YITCT': 'NMHC Y절편',
 'NOX_GRAD': 'NOX기울기',
 'NOX_YITCT': 'NOX Y절편',
 'PM_GRAD': 'PM기울기',
 'PM_YITCT': 'PM Y절편',
 'REG_DATETIME': '등록일시',
 'REG_ID': '등록아이디',
 'TEST_METHOD_CHK': '시험방법체크값',
 'TEST_MOD': '배출가스시험모드',
 'TEST_RST_SNO': '시험결과일련번호',
 'PN_GRAD': 'PN 기울기',
 'PN_YITCT': 'PN Y절편'}

In [114]:
cols_dict[name]['TEST_MOD']

'배출가스시험모드'

In [117]:
cols_df = pd.DataFrame({'컬럼명':cols_dict[name].keys(), '컬럼설명':cols_dict[name].values()})
cols_df

,컬럼명,컬럼설명
0,ADM_NO,관리번호
1,CERTI_MODI_SNO,변경인증일련번호
2,CO_GRAD,CO기울기
3,CO_YITCT,CO Y절편
4,MOD_DATETIME,수정일시
5,MOD_ID,수정아이디
6,NMHC_GRAD,NMHC기울기
7,NMHC_YITCT,NMHC Y절편
8,NOX_GRAD,NOX기울기
9,NOX_YITCT,NOX Y절편


In [120]:
temp = ct01.isnull().sum().reset_index().rename(columns={'index':'컬럼명', 0:'빈 값 개수'})
temp['총 행수'] = ct01.shape[0]
temp.insert(0, '번호', [x for x in range(1, temp.shape[0] + 1)])
ttemp = temp.merge(cols_df, on='컬럼명', how='left')
ttemp = ttemp[['번호', '컬럼명', '컬럼설명', '빈 값 개수', '총 행수']]
ttemp

,번호,컬럼명,컬럼설명,빈 값 개수,총 행수
0,1,ADM_NO,관리번호,0,411
1,2,CERTI_MODI_SNO,변경인증일련번호,0,411
2,3,TEST_METHOD_CHK,시험방법체크값,0,411
3,4,TEST_MOD,배출가스시험모드,0,411
4,5,TEST_RST_SNO,시험결과일련번호,0,411
5,6,NOX_GRAD,NOX기울기,244,411
6,7,NMHC_GRAD,NMHC기울기,258,411
7,8,CO_GRAD,CO기울기,248,411
8,9,PM_GRAD,PM기울기,245,411
9,10,NOX_YITCT,NOX Y절편,245,411


In [49]:
today_date = datetime.today().strftime('%Y.%m.%d')
today_date

'2023.10.30'

In [124]:
# 41.2s
with pd.ExcelWriter(os.path.join(an2_fold, f'건설기계_인증자료_데이터정보({today_date}).xlsx'), engine='openpyxl') as writer:
    for name in name_list:
        cols_df = pd.DataFrame({'컬럼명':cols_dict[name].keys(), '컬럼설명':cols_dict[name].values()})
        ct = construct_machine_certification_data_load(name)
        temp = ct.isnull().sum().reset_index().rename(columns={'index':'컬럼명', 0:'빈 값 개수'})
        temp['총 행수'] = ct.shape[0]
        temp.insert(0, '번호', [x for x in range(1, temp.shape[0] + 1)])
        ttemp = temp.merge(cols_df, on='컬럼명', how='left')
        ttemp = ttemp[['번호', '컬럼명', '컬럼설명', '빈 값 개수', '총 행수']]
        ttemp.to_excel(writer, sheet_name=f'{name}', index=False)

c:\Users\kbjung\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\kbjung\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\kbjung\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\kbjung\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\kbjung\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook con

## 과제2 > 건설기계 인증대장

## 전문정비(STD_TB_ERR_RES)

In [8]:
er_dict = {
    'DRVNG_DSTNC':'주행거리', 
    'VHRNO':'자동차등록번호', 
    'VIN':'차대번호', 
    'LOD_COX_MEVLU':'부하일산화탄소측정치', 
    'LOD_HC_MEVLU':'부하탄화수소측정치', 
    'LOD_NOX_MEVLU':'부하질소산화물측정치', 
    'LOD_SMO_MEVLU':'부하매연측정치', 
    'LOD_TQ_MEVLU':'부하토크측정치', 
    'OVHUL_YMD':'정비일자', 
    'SPCY_OVHUL_BIZPLC_NM':'전문정비사업소명', 
    'OVHUL_RCPT_NO':'정비접수번호', 
    'PRCINSP_RINSP_RCPT_NO':'정밀재검사접수번호', 
    'NOLOD_COX_MEVLU':'무부하일산화탄소측정치', 
    'NOLOD_HC_MEVLU':'무부하탄화수소측정치', 
    'NOLOD_AIR_EXCRT_MEVLU':'무부하공기과잉률측정치', 
    'NOLOD_SMO_MEVLU':'무부하매연측정치', 
}

In [3]:
# 0.3s
name = 'STD_TB_ERR_RES(20231011)'
file_name = f'{name}.csv'
er = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False)
er.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54617 entries, 0 to 54616
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   DRVNG_DSTNC            54617 non-null  float64
 1   VHRNO                  54617 non-null  object 
 2   VIN                    51051 non-null  object 
 3   LOD_COX_MEVLU          53884 non-null  object 
 4   LOD_HC_MEVLU           53886 non-null  object 
 5   LOD_NOX_MEVLU          53883 non-null  object 
 6   LOD_SMO_MEVLU          53941 non-null  object 
 7   LOD_TQ_MEVLU           53819 non-null  object 
 8   OVHUL_YMD              54617 non-null  int64  
 9   SPCY_OVHUL_BIZPLC_NM   54617 non-null  object 
 10  OVHUL_RCPT_NO          54617 non-null  int64  
 11  PRCINSP_RINSP_RCPT_NO  54617 non-null  object 
 12  NOLOD_COX_MEVLU        53870 non-null  object 
 13  NOLOD_HC_MEVLU         53871 non-null  object 
 14  NOLOD_AIR_EXCRT_MEVLU  28484 non-null  object 
 15  NO

In [9]:
er = er.rename(columns=er_dict)
er.columns

Index(['주행거리', '자동차등록번호', '차대번호', '부하일산화탄소측정치', '부하탄화수소측정치', '부하질소산화물측정치',
       '부하매연측정치', '부하토크측정치', '정비일자', '전문정비사업소명', '정비접수번호', '정밀재검사접수번호',
       '무부하일산화탄소측정치', '무부하탄화수소측정치', '무부하공기과잉률측정치', '무부하매연측정치'],
      dtype='object')

## 전문정비상세(STD_TB_ERR_RES_DET)

In [11]:
erd_dict = {
    'VHRNO':'자동차등록번호', 
    'VIN':'차대번호', 
    'OVHUL_CN':'정비내용', 
    'OVHUL_YMD':'정비일자', 
    'SPCY_OVHUL_BIZPLC_NM':'전문정비사업소명', 
    'OVHUL_CMPNT_NM':'정비부품명', 
    'OVHUL_RCPT_NO':'정비접수번호', 
    'PRCINSP_RINSP_RCPT_NO':'정밀재검사접수번호', 
}

In [4]:
# 0.3s
name = 'STD_TB_ERR_RES_DET(20231011)'
file_name = f'{name}.csv'
erd = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False)
erd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106819 entries, 0 to 106818
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VHRNO                  106818 non-null  object 
 1   VIN                    94670 non-null   object 
 2   OVHUL_CN               106817 non-null  object 
 3   OVHUL_YMD              106818 non-null  float64
 4   SPCY_OVHUL_BIZPLC_NM   106818 non-null  object 
 5   OVHUL_CMPNT_NM         106817 non-null  object 
 6   OVHUL_RCPT_NO          106818 non-null  float64
 7   PRCINSP_RINSP_RCPT_NO  0 non-null       float64
dtypes: float64(3), object(5)
memory usage: 6.5+ MB


In [12]:
erd = erd.rename(columns=erd_dict)
erd.columns

Index(['자동차등록번호', '차대번호', '정비내용', '정비일자', '전문정비사업소명', '정비부품명', '정비접수번호',
       '정밀재검사접수번호'],
      dtype='object')

## 구조변경검사(STD_DLM_TB_ERP_GUBUN_BAECHUL)

In [7]:
# 11.7s
name = 'STD_DLM_TB_ERP_GUBUN_BAECHUL(20231011)'
file_name = f'{name}.csv'
gb = pd.read_csv(os.path.join(raw_fold, file_name), low_memory=False)
gb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1321945 entries, 0 to 1321944
Data columns (total 39 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   AIR_PSSR                   1152822 non-null  float64
 1   AIR_TMPRT                  1152822 non-null  float64
 2   CO2_VAL                    1110507 non-null  float64
 3   DRVNG_DSTNC                1320563 non-null  float64
 4   FNCINSP_JGMT_YN            1321919 non-null  object 
 5   INSPOF_NM                  1321945 non-null  object 
 6   INSPOF_RPRSV_NM            1318198 non-null  object 
 7   MAXM_OTPT_MSRMT_VAL        445619 non-null   float64
 8   MAXM_OTPT_PRMT_VAL         544747 non-null   float64
 9   O2_VAL                     1110507 non-null  float64
 10  PRCINSP_YMD                1321945 non-null  int64  
 11  PRCINSP_VLD_YMD            127529 non-null   float64
 12  PRCINSP_VLD_WHL            1321945 non-null  object 
 13  PRCINSP_JGMT

# EDA

## 전문정비(STD_TB_ERR_RES)

In [17]:
er['정비일자'].isnull().sum()

0

In [15]:
er['정비일자'].describe()

count      54617.00
mean    20083896.95
std        16435.80
min     20060727.00
25%     20070801.00
50%     20080507.00
75%     20091109.00
max     20121010.00
Name: 정비일자, dtype: float64

In [19]:
er['정비일자'] = pd.to_datetime(er['정비일자'], format='%Y%m%d', errors='coerce')
er['정비일자'].head()

0   2008-01-10
1   2008-01-23
2   2008-01-08
3   2008-01-18
4   2008-01-07
Name: 정비일자, dtype: datetime64[ns]

In [20]:
er['정비일자'].describe()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_3124\328505498.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  er['정비일자'].describe()


count                   54617
unique                   1839
top       2007-10-02 00:00:00
freq                      123
first     2006-07-27 00:00:00
last      2012-10-10 00:00:00
Name: 정비일자, dtype: object

## 전문정비상세(STD_TB_ERR_RES_DET)

In [18]:
erd['정비일자'].isnull().sum()

1

In [16]:
erd['정비일자'].describe()

count     106818.00
mean    20083911.85
std        15852.89
min     20060727.00
25%     20070829.00
50%     20080529.00
75%     20090921.00
max     20120928.00
Name: 정비일자, dtype: float64

In [21]:
erd['정비일자'] = pd.to_datetime(erd['정비일자'], format='%Y%m%d', errors='coerce')
erd['정비일자'].head()

0   2007-02-08
1   2006-08-11
2   2006-08-11
3   2006-08-11
4   2006-12-11
Name: 정비일자, dtype: datetime64[ns]

In [22]:
erd['정비일자'].describe()

C:\Users\kbjung\AppData\Local\Temp\ipykernel_3124\338169835.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  erd['정비일자'].describe()


count                  106818
unique                   1832
top       2007-08-13 00:00:00
freq                      232
first     2006-07-27 00:00:00
last      2012-09-28 00:00:00
Name: 정비일자, dtype: object

## 구조변경검사(STD_DLM_TB_ERP_GUBUN_BAECHUL)